## 프림 알고리즘  (Prim's algorithm)

### 1. 프림 알고리즘이란?

- 프림 알고리즘 
  - 시작 정점을 선택한 후, 인접한 간선 중 최소 간선으로 연결된 정점을 선택하고, 해당 정점에서 다시 최소 간선으로 연결된 정점을 선택하는 방식
<br><br>

- Kruskal's algorithm vs Prim's algorithm
  - 공통점 : 탐욕 알고리즘을 기초로 함 (당장 눈 앞의 최소 비용을 선택해서, 결과적으로 최적의 솔루션을 찾음)
  - Kruskal's algorithm : 가중치가 작은 간선부터 선택하면서 MST를 구함
  - Prim's algorithm : 시작 정점을 선택한 후, 인접한 간선 중 최소 간선으로 연결된 정점을 선택하고, 해당 정점에서 다시 최소 간선으로 연결된 정점을 선택하는 방식으로 MST를 구함

### 2. 프림 알고리즘 과정 

<img src="https://www.fun-coding.org/00_Images/prim1.png" width=600>
<img src="https://www.fun-coding.org/00_Images/prim2.png" width=600>
<img src="https://www.fun-coding.org/00_Images/prim3.png" width=600>

1. 임의의 정점을 선택, '연결된 노드 집합(**connected_nodes**)'에 삽입
2. 선택된 정점에 연결된 간선들을 '간선 리스트(**candidate_edge_list**)'에 삽입
3. '간선 리스트(candidate_edge_list)'에서 최소 가중치를 가지는 간선부터 pop,
   - 해당 간선에 연결된 인접 정점이 '연결된 노드 집합'에 이미 들어 있다면, 스킵(사이클이 생기지 않도록)
   - 해당 간선에 연결된 인접 정점이 '연결된 노드 집합'에 들어 있지 않으면, 해당 간선을 선택하고, 해당 간선 정보를 '최소 신장 트리(**mst**)'에 삽입
4. '간선 리스트'에 더 이상의 간선이 없을 때까지 3번 반복

### 3. 프림 알고리즘 구현하기

In [1]:
myedges = [
    (7, 'A', 'B'), 
    (5, 'A', 'D'),
    (8, 'B', 'C'), 
    (9, 'B', 'D'), 
    (7, 'B', 'E'),
    (5, 'C', 'E'),
    (7, 'D', 'E'), 
    (6, 'D', 'F'),
    (8, 'E', 'F'), 
    (9, 'E', 'G'),
    (11,'F', 'G')
]

In [2]:
from collections import defaultdict
from heapq import *

def prim(start_node, edges):
    mst = list()
    
    # defaultdict 함수를 사용해서, key에 대한 value를 지정하지 않았을 시, 빈 리스트로 초기화하기
    adjacent_edges = defaultdict(list) 
    
    for weight, n1, n2 in edges:
        adjacent_edges[n1].append((weight, n1, n2))
        adjacent_edges[n2].append((weight, n2, n1))

    # 연결된 노드 집합(connected_nodes)
    connected_nodes = set(start_node)
    
    # 연결된 간선 리스트(candidate_edge_list) -> heap 구조로
    candidate_edge_list = adjacent_edges[start_node]
    heapify(candidate_edge_list)
    
    while candidate_edge_list:
        # 최소 가중치를 가지는 간선 추출
        weight, n1, n2 = heappop(candidate_edge_list)
        
        # 선택된 노드 넣어주기 
        if n2 not in connected_nodes:
            connected_nodes.add(n2)
            mst.append((weight, n1, n2))
            
            # 간선 (후보) 리스트에 간선 넣어주기 
            for edge in adjacent_edges[n2]:
                if edge[2] not in connected_nodes:
                    heappush(candidate_edge_list, edge)

    return mst

In [3]:
prim ('A', myedges)

[(5, 'A', 'D'),
 (6, 'D', 'F'),
 (7, 'A', 'B'),
 (7, 'B', 'E'),
 (5, 'E', 'C'),
 (9, 'E', 'G')]

### 4. 프림 알고리즘 시간 복잡도
- 시간 복잡도 : O($ElogE$)
  - 최악의 경우, while 구문에서 모든 간선에 대해 반복하고, 최소 힙 구조를 사용

## 개선된 프림 알고리즘 (Improved Prim's algorithm)

### 5. (참고) 개선된 프림 알고리즘 

- 간선이 아닌 **노드를 중심**으로 우선순위 큐를 적용하는 방식

### 6. 개선된 프림 알고리즘 과정

- 우선순위 큐 **keys**에 특정 정점의 key 값은 0, 이외 정점들의 key값은 무한대로 넣음
- Extract Min 로직
    - key 값이 가장 작은 {정점:key} pop
- Decrease key 로직
    - 해당 정점의 인접 정점들에 대해, 연결된 가중치 값과 key 값을 비교하여, 가중치 값이 작으면 해당 key 값 갱신
    - key 값 갱신 시, 우선순위 큐는 최소 key 값을 가지는 {정점:key}를 루트 노드로 올려놓도록 재구성

### 7. 개선된 프림 알고리즘 구현하기

In [4]:
# 우선순위 큐(최소힙) 구조에서, 이미 들어가 있는 데이터의 값 변경시, 최소값을 가지는 데이터를 루트노드로 올려놓도록 재구성하는 기능이 필요
# heapdict 라이브러리를 통해, 해당 기능을 간단히 구현
from heapdict import heapdict

def prim(graph, start):
    
    mst = list()
    keys = heapdict() # 정점 정보 담고 있음 
    pi = dict()       # 어디 정점으로 부터 왔는지 정보 저장
    total_weight = 0
    
    for node in graph.keys():
        keys[node] = float('inf')
        pi[node] = None
    
    keys[start] = 0
    pi[start] = start

    while keys:
        current_node, current_key = keys.popitem()
        mst.append([pi[current_node], current_node, current_key])
        
        total_weight += current_key
        
        for adjacent, weight in mygraph[current_node].items():
            if adjacent in keys and weight < keys[adjacent]:
                keys[adjacent] = weight
                pi[adjacent] = current_node
                
    return mst, total_weight

In [5]:
mygraph = {
    'A': {'B': 7, 'D': 5},
    'B': {'A': 7, 'D': 9, 'C': 8, 'E': 7},
    'C': {'B': 8, 'E': 5},
    'D': {'A': 5, 'B': 9, 'E': 7, 'F': 6},
    'E': {'B': 7, 'C': 5, 'D': 7, 'F': 8, 'G': 9},
    'F': {'D': 6, 'E': 8, 'G': 11},
    'G': {'E': 9, 'F': 11}    
}

mst, total_weight = prim(mygraph, 'A')

print ('MST:', mst)
print ('Total Weight:', total_weight)

MST: [['A', 'A', 0], ['A', 'D', 5], ['D', 'F', 6], ['A', 'B', 7], ['D', 'E', 7], ['E', 'C', 5], ['E', 'G', 9]]
Total Weight: 39


### 8. 개선된 프림 알고리즘 시간 복잡도
- 시간 복잡도: $ O(ElogV) $
<br><br>

- 최초 key 생성 시간 복잡도: $ O(V) $
- while 구문과 keys.popitem() 의 시간 복잡도는 $ O(VlogV) $
  - while 구문은 V 번 실행됨
  - heap 에서 최소 key 값을 가지는 노드 정보 추출 시(pop)의 시간 복잡도: $ O(logV) $
- for 구문의 총 시간 복잡도는 $ O(ElogV) $
  - for 구문은 while 구문 반복시에 결과적으로 총 최대 간선의 수 E 만큼 실행 가능 $ O(E) $
  - for 구문 안에서 key값 변경시마다 heap 구조를 변경해야 하며, heap 에는 최대 V 개의 정보가 있으므로 $ O(logV) $

- 따라서, 총 시간 복잡도는 $ O(V + VlogV + ElogV) $ 이며,
  - O(V)는 전체 시간 복잡도에 큰 영향을 미치지 않으므로 삭제,
  - E > V 이므로 (최대 $ V^2 = E $ 가 될 수 있음), $ O((V + E)logV) $ 는 간단하게 $ O(ElogV) $ 로 나타낼 수 있음